<a href="https://colab.research.google.com/github/karl-gardner/droplet_detection/blob/master/yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://user-images.githubusercontent.com/26833433/99805971-90f66b80-2b3d-11eb-80eb-8b45a15cb68e.jpg">

This is the **official YOLOv3 🚀 notebook** authored by **Ultralytics**, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/ultralytics/yolov3 and https://www.ultralytics.com. Thank you!

# 0. Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
!git clone https://github.com/karl-gardner/droplet_detection  # clone repo
%cd /content/droplet_detection/yolov3
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from utils.general import xywhn2xyxy
import random
from google.colab import files
import csv

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla P100-PCIE-16GB)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.1 Curl droplet dataset from roboflow (PC3DropletDetection2)

Data With Augmentation for Training (final_dataset)

In [3]:
%cd /content/droplet_detection
!curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
clear_output()

Data with No Augmentation (No_Augmentation)

In [4]:
# %cd /content/droplet_detection
# !curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# clear_output()

# 1.2 Curl cell model dataset from roboflow (Cropped_Drops2)

Data With Augmentation for Training (final_dataset)

In [5]:
# %cd /content/droplet_detection
# !curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# clear_output()

Data with No Augmentation (No_Augmentation)

In [6]:
# %cd /content/droplet_detection
# !curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# clear_output()

# 2. Get information for annotations: Table ann


For droplet model

In [7]:
# %cd /content/droplet_detection
# import funcs

# # drop_labels returns a list of the counted labels: [drop_0cell, drop_1cell, drop_2cell, drop_3cell]
# totals = []

# totals.append(funcs.drop_labels('train/labels','Train'))

# totals.append(funcs.drop_labels('valid/labels','Validation'))

# totals.append(funcs.drop_labels('test/labels','Test'))

# totals = np.array(totals)

# print("Total Count")
# print("drop_0cell: " + str(np.sum(totals[:,0])))
# print("drop_1cell: " + str(np.sum(totals[:,1])))
# print("drop_2cell: " + str(np.sum(totals[:,2])))
# print("drop_3cell: " + str(np.sum(totals[:,3])))
# print("combined: " + str(np.sum(totals[:,0:4])))
# print("images: " + str(np.sum(totals[:,4])))

For cell model

In [8]:
# %cd /content/droplet_detection
# import funcs

# # cell_labels returns a list of the counted labels: [drop_0cell, drop_1cell, drop_2cell, drop_3cell]
# totals = []

# totals.append(funcs.cell_labels('train/labels','Train'))

# totals.append(funcs.cell_labels('valid/labels','Validation'))

# totals.append(funcs.cell_labels('test/labels','Test'))

# totals = np.array(totals)

# print("Total Count")
# print("combined: " + str(np.sum(totals[:,0])))
# print("images: " + str(np.sum(totals[:,1])))

# 3. Save Images for Annotation Examples: Figure ann

In [9]:
# def box_label(box, thick, label='', color=(128, 128, 128), txt_color=(0, 0, 0)):
#   p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
#   cv2.rectangle(im, p1, p2, color, thickness=thick, lineType=cv2.LINE_AA)
#   if label:
#       tf = max(1 - 1, 1)  # font thickness
#       w, h = cv2.getTextSize(label, 0, fontScale=1 / 3, thickness=tf)[0]  # text width, height
#       outside = p1[1] - h - 3 >= 0  # label fits outside box
#       p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
#       cv2.rectangle(im, p1, p2, color, -1, cv2.LINE_AA)  # filled
#       cv2.putText(im, label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2), 0, 1 / 3, txt_color,
#                   thickness=tf, lineType=cv2.LINE_AA)

In [10]:
# !rm -r /annotation_examples
# !mkdir /annotation_examples
# %cd /content/drive/MyDrive/droplet_classification/data_files


# for f in os.listdir("annotation_examples/droplet_model"):
#   if f.endswith(".jpg"):
#     boxes = []
#     classes = []
#     with open("annotation_examples/droplet_model/"+str(f[:-4]+".txt")) as labels:
#       for line in labels.readlines():
#         line = line.split()
#         boxes.append([float(line[1]),float(line[2]),float(line[3]),float(line[4])])
#         classes.append(int(line[0]))
#     boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#     im = cv2.imread("annotation_examples/droplet_model/" + f)
#     !mkdir /annotation_examples/{f[:-4]}
#     for i in range(boxes.shape[0]):
#       cv2.imwrite("/annotation_examples/"+f[:-4]+"/box_"+str(i)+".png", im[int(boxes[i][1]):int(boxes[i][3]), int(boxes[i][0]):int(boxes[i][2])])
#     for i in range(boxes.shape[0]):
#       class_label = classes[i]
#       if class_label == 0:
#         lab = "drop_0cell"
#         col = (0,0,255)
#       elif class_label == 1:
#         lab = "drop_1cell"
#         col = (0,255,255)
#       elif class_label == 2:
#         lab = "drop_2cell"
#         col = (255,0,127)
#       else:
#         lab = "drop_3cell"
#         col = (255,0,255) 
#       box_label(boxes[i,:],1 ,lab,col)
#       cv2.imwrite("/annotation_examples/"+f[:-4]+".png",im)


# for f in os.listdir("annotation_examples/cell_model"):
#   if f.endswith(".jpg"):
#     boxes = []
#     with open("annotation_examples/cell_model/"+str(f[:-4]+".txt")) as labels:
#       for line in labels.readlines():
#         line = line.split()
#         boxes.append([float(line[1]),float(line[2]),float(line[3]),float(line[4])])
#     boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#     im = cv2.imread("annotation_examples/cell_model/" + f)
#     for i in range(boxes.shape[0]):
#       box_label(boxes[i,:], 2,color=(0,0,255))
#     cv2.imwrite("/annotation_examples/"+f[:-4]+".png",im)


# !zip -r /annotation_examples.zip /annotation_examples
# clear_output()
# files.download("/annotation_examples.zip")

# 4. Save cropped droplets with one or more cells

In [11]:
# # Now convert ground truth labels and boxes
# %cd /content/droplet_detection/yolov3
# !rm -rf /cropped_drops
# !rm /annotate_cropped.zip
# !mkdir /cropped_drops

# # Using the un-augmented dataset save around 2000 images from training, validation, and test droplets with 1 or more cells in them

# counter_tot = 0
# # First the training set
# counter_set = 0
# for j, im_file in enumerate(os.listdir("../train/images")):
#   if j % 5 == 0:
#     f_label = im_file[0:-4]+".txt"
#     with open("../train/labels/"+f_label) as f:
#       lines = f.readlines()
#       rows = len(lines)
#       boxes = []
#       for line in lines:
#         line = line.split()
#         if int(line[0]) == 0:
#           continue
#         x = float(line[1])
#         y = float(line[2])
#         mean_wh = (float(line[3])+float(line[4]))/2
#         if x + mean_wh/2 > 1:
#           x = 1 - mean_wh/2
#         if y + mean_wh/2 > 1:
#           y = 1 - mean_wh/2
#         if x-mean_wh/2 < 0:
#           x = mean_wh/2
#         if y-mean_wh/2 < 0:
#           y = mean_wh/2
#         boxes.append([x,y,mean_wh,mean_wh])
#     boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#     im = cv2.imread("../train/images/"+im_file)
#     for i in range(boxes.shape[0]):
#       # May not be square by one pixel... make square
#       if int(boxes[i,3])-int(boxes[i,1]) < int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,3] += 1
#       if int(boxes[i,3])-int(boxes[i,1]) > int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,2] += 1
#       cropped = im[int(boxes[i,1]):int(boxes[i,3]),int(boxes[i,0]):int(boxes[i,2]),:]
#       cv2.imwrite("/cropped_drops/im_"+str(counter_tot)+".png",cropped)
#       counter_tot += 1
#       counter_set += 1
# print("number of images saved from train set: ",counter_set)

# # Now the validation set
# counter_set = 0
# for j, im_file in enumerate(os.listdir("../valid/images")):
#   if j % 5 == 0:
#     f_label = im_file[0:-4]+".txt"
#     with open("../valid/labels/"+f_label) as f:
#       lines = f.readlines()
#       rows = len(lines)
#       boxes = []
#       for line in lines:
#         line = line.split()
#         if int(line[0]) == 0:
#           continue
#         x = float(line[1])
#         y = float(line[2])
#         mean_wh = (float(line[3])+float(line[4]))/2
#         if x + mean_wh/2 > 1:
#           x = 1 - mean_wh/2
#         if y + mean_wh/2 > 1:
#           y = 1 - mean_wh/2
#         if x-mean_wh/2 < 0:
#           x = mean_wh/2
#         if y-mean_wh/2 < 0:
#           y = mean_wh/2
#         boxes.append([x,y,mean_wh,mean_wh])
#     boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#     im = cv2.imread("../valid/images/"+im_file)
#     for i in range(boxes.shape[0]):
#       # May not be square by one pixel... make square
#       if int(boxes[i,3])-int(boxes[i,1]) < int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,3] += 1
#       if int(boxes[i,3])-int(boxes[i,1]) > int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,2] += 1
#       cropped = im[int(boxes[i,1]):int(boxes[i,3]),int(boxes[i,0]):int(boxes[i,2]),:]
#       cv2.imwrite("/cropped_drops/im_"+str(counter_tot)+".png",cropped)
#       counter_tot += 1
#       counter_set += 1
# print("number of images saved from validation set: ",counter_set)


# # Now the test set
# counter_tot = 0
# counter_set = 0
# for im_file in enumerate(os.listdir("../test/images")):
#   if j % 5 == 0:
#     f_label = im_file[0:-4]+".txt"
#     with open("../test/labels/"+f_label) as f:
#       lines = f.readlines()
#       rows = len(lines)
#       boxes = []
#       for line in lines:
#         line = line.split()
#         if int(line[0]) == 0:
#           continue
#         x = float(line[1])
#         y = float(line[2])
#         mean_wh = (float(line[3])+float(line[4]))/2
#         if x + mean_wh/2 > 1:
#           x = 1 - mean_wh/2
#         if y + mean_wh/2 > 1:
#           y = 1 - mean_wh/2
#         if x-mean_wh/2 < 0:
#           x = mean_wh/2
#         if y-mean_wh/2 < 0:
#           y = mean_wh/2
#         boxes.append([x,y,mean_wh,mean_wh])
#     boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#     im = cv2.imread("../test/images/"+im_file)
#     for i in range(boxes.shape[0]):
#       # May not be square by one pixel... make square
#       if int(boxes[i,3])-int(boxes[i,1]) < int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,3] += 1
#       if int(boxes[i,3])-int(boxes[i,1]) > int(boxes[i,2])-int(boxes[i,0]):
#         boxes[i,2] += 1
#       cropped = im[int(boxes[i,1]):int(boxes[i,3]),int(boxes[i,0]):int(boxes[i,2]),:]
#       cv2.imwrite("/cropped_drops/im_"+str(counter_tot)+".png",cropped)
#       counter_tot += 1
#       counter_set += 1
# print("number of images saved from test set: ",counter_set)
# print("Number of total images saved from train, validation, and test sets: ",counter_tot)

# !zip -r /annotate_cropped.zip /cropped_drops
# clear_output()
# files.download("/annotate_cropped.zip")

# 5.1 Train

Download [COCO128](https://www.kaggle.com/ultralytics/coco128), a small 128-image tutorial dataset, start tensorboard and train YOLOv3 from a pretrained checkpoint for 3 epochs (note actual training is typically much longer, around **300-1000 epochs**, depending on your dataset).

Train a YOLOv3 model on [COCO128](https://www.kaggle.com/ultralytics/coco128) with `--data coco128.yaml`, starting from pretrained `--weights yolov3.pt`, or from randomly initialized `--weights '' --cfg yolov3.yaml`. Models are downloaded automatically from the [latest YOLOv3 release](https://github.com/ultralytics/yolov3/releases), and **COCO, COCO128, and VOC datasets are downloaded automatically** on first use.

All training results are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.


In [12]:
# # Train YOLOv3 on COCO128 for 3 epochs
# %cd /content/droplet_detection/yolov3
# !python train.py --img 544 --batch 32 --epochs 200 --data ../yaml_files/cell_model.yaml --weights '' --cfg models/yolov3.yaml --cache

In [13]:
# %cp /content/droplet_detection/yolov3/runs/train/exp/weights/best.pt /content/drive/MyDrive/droplet_classification/data_files/yolov3_cell_weights_46_best.pt

# 5.2 Generate high quality mAP plots for validation set

In [14]:
# map5 = []
# map595 = []
# with open('/content/drive/MyDrive/droplet_classification/data_files/yolov3_train_results.csv', newline='') as csvfile:
#   spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#   next(spamreader)
#   for row in spamreader:
#     for i, elem in enumerate(row):
#       if i == 6:
#         map5.append(float(elem.strip()))
#       if i == 7:
#         map595.append(float(elem.strip()))
# fig, axs = plt.subplots(1, 2, figsize=(15,6))
# axs[0].plot(map5, color="blue", marker='.')
# axs[0].set_ylim(0, 1)
# axs[0].tick_params(axis='both', which='major', labelsize=29)
# axs[0].set_xlabel("Epochs", fontsize=35)
# axs[0].set_ylabel("mAP @ IOU 0.5", fontsize=35)
# axs[1].plot(map595, color="blue", marker='.')
# axs[1].set_ylim(0, 1)
# axs[1].tick_params(axis='both', which='major', labelsize=29)
# axs[1].set_ylabel("mAP\n@ IOU 0.5:0.95", fontsize=35)
# axs[1].set_xlabel("Epochs", fontsize=35)
# fig.suptitle("YOLOv3",fontsize=45, y=1)
# fig.tight_layout(pad=4)
# fig.subplots_adjust(top=0.88)

# fig.savefig("/mAP_yolov3.png", dpi=500)
# files.download("/mAP_yolov3.png")

# 6.1 mAP calculation for droplet model test set: Table drop

In [15]:
# Run YOLOv3 on COCO test-dev2017 using --task test
%cd /content/droplet_detection/yolov3
!python val.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --data ../yaml_files/droplet_model.yaml --task test

# # Download Figures
# files.download("runs/val/exp/PR_curve.png")
# files.download("runs/val/exp/legend.png")

/content/droplet_detection/yolov3
val: data=../yaml_files/droplet_model.yaml, weights=['/content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt'], batch_size=32, imgsz=544, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv3 🚀 2716a2a torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 261 layers, 61513585 parameters, 0 gradients, 154.8 GFLOPs
test: Scanning '../test/labels' images and labels...70 found, 0 missing, 0 empty, 0 corrupted: 100% 70/70 [00:00<00:00, 1183.69it/s]
test: New cache created: ../test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:05<00:00,  1.69s/it]
                 all         70       1653      0.935      0.945      0.974      0.917
      

# 6.2 mAP calculation for cell model test set: Table cell

In [16]:
# # Run YOLOv3 on COCO test-dev2017 using --task test
# %cd /content/droplet_detection/yolov3
# !rm -rf runs/val/exp
# !python val.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_cell_46_best.pt --img 544 --data /content/droplet_detection/yaml_files/cell_model.yaml --task test

# 7.1 Plot ground truth labels vs predictions for droplet model test set: Figure gvpdrop

In [17]:
# %cd /content/droplet_detection/yolov3
# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source ../test/images --save-txt --save-conf
# clear_output()

# %cd /content/droplet_detection
# import funcs
# !rm -r /test_results
# funcs.save_results('test/images', 'yolov3', 'droplet')

# image_files = sorted(os.listdir("/test_results/gts"))
# image_files = [image_files[27], image_files[60]]
# for image_file in image_files:
#   files.download("/test_results/gts/" + image_file)
#   files.download("/test_results/preds/" + image_file)

# 7.2 Plot ground truth labels vs predictions for cell model test set: Figure gvpdrop

In [18]:
# %cd /content/droplet_detection/yolov3
# !rm -r runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_cell_46_best.pt --img 544 --source ../test/images --save-txt --save-conf
# clear_output()

# %cd /content/droplet_detection
# import funcs
# !rm -r /test_results
# funcs.save_results('test/images', 'yolov3', 'cell')

# random.seed(7)
# image_files = os.listdir("/test_results/gt_vs_pred")
# image_files = random.sample(image_files, 9)
# for image_file in image_files:
#   files.download("/test_results/gt_vs_pred/" + image_file)

# 8. Conduct average run times across test set to get FPS

In [19]:
# %cd /content/droplet_detection/yolov3
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_cell_46_best.pt --img 544 --conf-thres 0.6 --source=../test/images

# Computer Statistics

In [20]:
# !nvidia-smi -L
# !nvidia-smi

In [21]:
# !lscpu |grep 'Model name'

# #no.of sockets i.e available slots for physical processors
# !lscpu | grep 'Socket(s):'

# #no.of cores each processor is having 
# !lscpu | grep 'Core(s) per socket:'

# #no.of threads each core is having
# !lscpu | grep 'Thread(s) per core'

# !lscpu | grep "L3 cache" 

# #if it had turbo boost it would've shown Min and Max MHz also but it is only showing current frequency this means it always operates at shown frequency
# !lscpu | grep "MHz"

# #memory that we can use
# !free -h --si | awk  '/Mem:/{print $2}'

# #hard disk space that we can use
# !df -h / | awk '{print $4}'

# 9.1 Compare Production Set with Poisson Distribution: Figure pcomp

For second trial of images between 0-6 minutes (11/9/2021)

In [22]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_0-6"))
# production_images = production_images_all[9000:15400]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_0-6/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML = classes.count(0)
# one_ML = classes.count(1)
# two_ML = classes.count(2)
# three_ML = classes.count(3)
# tot_ML = zero_ML + one_ML + two_ML + three_ML

For second trial of images between 87-89 minutes (11/9/2021)

In [23]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_87-89"))
# production_images = production_images_all[0:6400]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_87-89/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML_2 = classes.count(0)
# one_ML_2 = classes.count(1)
# two_ML_2 = classes.count(2)
# three_ML_2 = classes.count(3)
# tot_ML_2 = zero_ML_2 + one_ML_2 + two_ML_2 + three_ML_2

For third trial of images between 0-3 minutes (11/15/2021)

In [24]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3"))
# production_images = production_images_all[3000:9400]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML = classes.count(0)
# one_ML = classes.count(1)
# two_ML = classes.count(2)
# three_ML = classes.count(3)
# tot_ML = zero_ML + one_ML + two_ML + three_ML

For third trial of images between 59-60 minutes (11/15/2021)

In [25]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_59-60"))
# production_images = production_images_all[0:6400]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_59-60/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML_2 = classes.count(0)
# one_ML_2 = classes.count(1)
# two_ML_2 = classes.count(2)
# three_ML_2 = classes.count(3)
# tot_ML_2 = zero_ML_2 + one_ML_2 + two_ML_2 + three_ML_2

Plot Poisson Distribution Curves

In [26]:
# c_0 = 7e6
# d = 74e-6
# v_0 = (4/3)*math.pi*(d/2)**3
# v_0 = v_0*10**6

# # Calculate lambda as the expected value or the average number of cells per nanoliter drop
# lam = c_0*v_0

# Pr = []
# K = [0,1,2]
# for k in K:
#   Pr.append((lam**k)*math.exp(-1*lam)/math.factorial(k))
# K.append(3)
# Pr.append(1-sum(Pr))

# plt.plot(K,[zero_ML/tot_ML,one_ML/tot_ML,two_ML/tot_ML,three_ML/tot_ML],linestyle='--', marker='o', color='r', label="YOLOv3 Model " u"\u2248" " 0-64 seconds")
# # plt.plot(K,[zero_ML_2/tot_ML_2,one_ML_2/tot_ML_2,two_ML_2/tot_ML_2,three_ML_2/tot_ML_2],linestyle='--', marker='o', color='maroon', label="YOLOv3 Model " u"\u2248" " 5220-5284 seconds")
# plt.plot(K,Pr, marker='o', color='b', label='Poisson Distribution')


# plt.xticks(K,("0","1","2",">3"))
# plt.xlabel("k")
# # plt.ylabel("P(x=k)")
# plt.ylabel("% of Droplets")
# # plt.title("Trial 1: 12800 images")

# plt.legend()

# plt.savefig("/trial_1.png",dpi=500)
# files.download("/trial_1.png")

# 9.2 Compare Production Set with Hand Counted Percentages: Figure hcomp

Run 500 images from second trial and compare droplet ratios with hand counting (Trial 1)

In [27]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_0-6"))
# production_images = production_images_all[1000:1500]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-9-2021_t02_0-6/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# # !zip -r /hand_count.zip /content/droplet_detection/yolov3/data/production_images
# # files.download("/hand_count.zip")

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML = classes.count(0)
# one_ML = classes.count(1)
# two_ML = classes.count(2)
# three_ML = classes.count(3)
# tot_ML = zero_ML + one_ML + two_ML + three_ML

# # For hand counting of 500 images
# zero_BH = 1993
# one_BH = 3399
# two_BH = 2658
# three_BH = 3530
# tot_BH = zero_BH + one_BH + two_BH + three_BH

Run 500 images from third trial and compare droplet ratios with hand counting (Trial 2)

In [28]:
# %cd /content/droplet_detection/yolov3
# production_images_all = sorted(os.listdir("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3"))
# production_images = production_images_all[1000:1500]


# !rm -rf /content/droplet_detection/yolov3/data/production_images
# !mkdir /content/droplet_detection/yolov3/data/production_images
# for image in production_images:
#   !cp /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3/{image} /content/droplet_detection/yolov3/data/production_images/{image}

# # !zip -r /hand_count.zip /content/droplet_detection/yolov3/data/production_images
# # files.download("/hand_count.zip")

# !rm -rf runs/detect/exp
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/droplet_detection/yolov3/data/production_images --save-txt --line-thickness 1

# classes = []
# for f in os.listdir('runs/detect/exp/labels'):
#   with open('runs/detect/exp/labels/' + f) as read_file:
#       lines = read_file.readlines()
#       for line in lines:
#         line = line.split()
#         classes.append(int(line[0]))
# zero_ML = classes.count(0)
# one_ML = classes.count(1)
# two_ML = classes.count(2)
# three_ML = classes.count(3)
# tot_ML = zero_ML + one_ML + two_ML + three_ML

# # For hand counting of 500 images
# zero_BH = 1610
# one_BH = 3068
# two_BH = 3827
# three_BH = 3785
# tot_BH = zero_BH + one_BH + two_BH + three_BH

Plot ML vs Hand Distribution Curves

In [29]:
# K = [0,1,2,3]
# plt.plot(K,[zero_ML/tot_ML,one_ML/tot_ML,two_ML/tot_ML,three_ML/tot_ML],linestyle='--', marker='o', color='r', label="YOLOv3 Model")
# plt.plot(K,[zero_BH/tot_BH,one_BH/tot_BH,two_BH/tot_BH,three_BH/tot_BH],linestyle='--', marker='o', color='g', label='Hand Counted')

# plt.xticks(K,("0","1","2",">3"))
# plt.xlabel("k")
# # plt.ylabel("P(x=k)")
# plt.ylabel("% of Droplets")
# plt.title("Trial 2: " u"\u2248" " 0-5 seconds (500 images)")

# plt.legend(loc='lower right')

# plt.savefig("/trial_2.png",dpi=500)
# files.download("/trial_2.png")

# 10.1 Conduct inference on production set with both models: Figure ework

In [30]:
# %cd /content/droplet_detection/yolov3
# !rm -r runs/cropped_drops
# !rm -r runs/detect/exp
# !rm -r runs/detect/exp2

# prod_image = "test_03002_Cam_V710_Cine1.png"
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3/{prod_image} --save-txt --line-thickness 2 --hide-labels --hide-conf
# files.download("runs/detect/exp/" + prod_image)

# !mkdir runs/cropped_drops
# counter = 0
# with open("runs/detect/exp/labels/"+prod_image[:-4]+".txt") as f:
#   boxes = []
#   for i, line in enumerate(f.readlines()):
#     line = line.split()
#     x = float(line[1])
#     y = float(line[2])
#     mean_wh = (float(line[3])+float(line[4]))/2
#     if x + mean_wh/2 > 1:
#       x = 1 - mean_wh/2
#     if y + mean_wh/2 > 1:
#       y = 1 - mean_wh/2
#     if x-mean_wh/2 < 0:
#       x = mean_wh/2
#     if y-mean_wh/2 < 0:
#       y = mean_wh/2
#     boxes.append([x,y,mean_wh,mean_wh])
#   boxes = xywhn2xyxy(np.array(boxes), w=544, h=544)
#   im = cv2.imread("/content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3/"+prod_image)
#   for i in range(boxes.shape[0]):
#     # May not be square by one pixel... make square
#     if int(boxes[i,3])-int(boxes[i,1]) < int(boxes[i,2])-int(boxes[i,0]):
#       boxes[i,3] += 1
#     if int(boxes[i,3])-int(boxes[i,1]) > int(boxes[i,2])-int(boxes[i,0]):
#       boxes[i,2] += 1
#     cropped_resized = cv2.resize(im[int(boxes[i,1]):int(boxes[i,3]),int(boxes[i,0]):int(boxes[i,2])],(544, 544))
#     cv2.imwrite("runs/cropped_drops/cropped_drop_"+str(counter)+".png",cropped_resized)
#     counter += 1

# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_cell_46_best.pt --img 544 --conf-thres 0.6 --source runs/cropped_drops --line-thickness 2 --hide-labels --hide-conf
# !zip -r /cropped_pred.zip /content/droplet_detection/yolov3/runs/detect/exp2
# files.download("/cropped_pred.zip")

In [31]:
# """
# The paper was already written and model was trained, however this would probably have been the better option
# to train and predict the cropped droplets... shoot maybe next round I will do this but this would have been cool
# to add in the paper!  The idea is to pad the extra height or with with the average pixel value from the image. 
# This way I am not taking some of another droplet with a half droplet that is cropped on the edge of the image.
# Still works for detecting though! 
# """

# %cd /content/droplet_detection/yolov3
# !rm -r runs/cropped_drops
# !rm -r runs/detect/exp
# !rm -r runs/detect/exp2


# prod_image = "test_03002_Cam_V710_Cine1.png"
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_243_best.pt --img 544 --conf-thres 0.6 --source /content/drive/MyDrive/droplet_classification/data_files/production_set/data_11-15-2021_t03_0-3/{prod_image} --line-thickness 2 --hide-labels --hide-conf --save-crop
# files.download("runs/detect/exp/" + prod_image)

# count = 0
# !mkdir runs/cropped_drops
# for i in range(4):
#   for im_file in os.listdir('runs/detect/exp/crops/drop_{}cell'.format(i)):
#     image = cv2.imread('runs/detect/exp/crops/drop_{}cell/{}'.format(i,im_file))
#     mean = np.mean(image)
#     h = image.shape[0]
#     w = image.shape[1]
#     if h > w:
#       padded = np.zeros((h, h, 3))
#       l = int((h-w)/2)
#       padded[:,0:l,:] = mean
#       padded[:,l:l+w,:] = image
#       padded[:,l+w:,:] = mean
#     elif w > h:
#       padded = np.zeros((w, w, 3))
#       l = int((w-h)/2)
#       padded[0:l,:,:] = mean
#       padded[l:l+h,:,:] = image
#       padded[l+h:,:,:] = mean
#     else:
#       padded = image.copy()

#     cv2.imwrite('runs/cropped_drops/im_{}.png'.format(count), cv2.resize(padded,(544,544)))
#     count += 1

# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov3_weights_cell_46_best.pt --img 544 --conf-thres 0.6 --source runs/cropped_drops --hide-labels --hide-conf


# 10.2 Construct Poisson Curve for multiple lambdas: Figure ework

In [32]:
# Prs = []
# lambdas = [1,3,6,9]
# colors = ["r","g","b","black",]
# K = list(range(0,21,1))

# for lam,c in zip(lambdas,colors):
#   Pr = []
#   for k in K:
#     Pr.append((lam**k)*math.exp(-1*lam)/math.factorial(k))
  
#   plt.plot(K,Pr, markersize=3.5,marker='o', color=c, label="\u03BB = " + str(lam))

# plt.xticks([0,5,10,15,20],["0","5","10","15","20"])
# plt.yticks([0.0,0.1,0.2,0.3,0.4],["0.0","0.1","0.2","0.2","0.3","0.4"])
# plt.xlabel("k")
# plt.ylabel("Pr(X=k)")

# plt.legend()

# plt.savefig("/poisson_distribution.png",dpi=500)
# files.download("/poisson_distribution.png")